# Rossman Time-Series Forecasting

This notebook only contains the forecasting part  
The feature engineering is in a separate notebook and saved to the following files
   -  "PATH/joined"
   -  "PATH/joined_test"

In [1]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)
PATH='data/rossmann/'

In [2]:
joined = pd.read_feather(f'{PATH}/joined')
joined_test = pd.read_feather(f'{PATH}joined_test')

In [3]:
joined.columns

Index(['index', 'Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open',
       'Promo', 'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week',
       'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
       'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start',
       'Elapsed', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'State', 'file',
       'week', 'trend', 'file_DE', 'week_DE', 'trend_DE', 'Date_DE',
       'State_DE', 'Month_DE', 'Day_DE', 'Dayofweek_DE', 'Dayofyear_DE',
       'Is_month_end_DE', 'Is_month_start_DE', 'Is_quarter_end_DE',
       'Is_quarter_start_DE', 'Is_year_end_DE', 'Is_year_start_DE',
       'Elapsed_DE', 'Max_TemperatureC', 'Mean_TemperatureC',
       'Min_TemperatureC', 'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC',
       'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
      

# Prepare for DL

from here on take it as given that all the above has been done

In [4]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw']

contin_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']

n = len(joined); n

844338

In [5]:
dep = 'Sales'
joined = joined[cat_vars+contin_vars+[dep, 'Date']].copy()
joined_test[dep] = 0
joined_test = joined_test[cat_vars+contin_vars+[dep, 'Date', 'Id']].copy()

# change columns into categories
# Loop through cat_vars and turn applicable data frame columns into categorical columns.
for v in cat_vars: joined[v] = joined[v].astype('category').cat.as_ordered()
apply_cats(joined_test, joined)

# change continuous variables into 32-bit floating point
# PyTorch expects 32-bt floating point
# 1's and 0's will be 32-bt floating point
for v in contin_vars:
    joined[v] = joined[v].fillna(0).astype('float32')
    joined_test[v] = joined_test[v].fillna(0).astype('float32')
    
#samp_size = n
joined_samp = joined.set_index("Date")

We can now process our data... Notice that pandas still displays categories as strings (e.g., StoreType). Even though we set some of the columns as “category” (e.g. ‘StoreType’, ‘Year’), Pandas still display as string in the notebook.

### Proc_df

In [6]:
df, y, nas, mapper = proc_df(joined_samp, 'Sales', do_scale=True)
yl = np.log(y)

joined_test = joined_test.set_index("Date")

df_test, _, nas, mapper = proc_df(joined_test, 'Sales', do_scale=True, skip_flds=['Id'],
                                  mapper=mapper, na_dict=nas)

val_idx = np.flatnonzero(
    (df.index<=datetime.datetime(2014,9,17)) & (df.index>=datetime.datetime(2014,8,1)))

# log y

When you take the log of the data, getting the root mean squared error will actually get you the root mean square percentage error.

In [7]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

# Tree Model

In [8]:
from sklearn.ensemble import RandomForestRegressor

In [11]:
((val,trn), (y_val,y_trn)) = split_by_idx(val_idx, df.values, yl)
mrf = RandomForestRegressor(n_estimators=40, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)
%time mrf.fit(trn, y_trn);

preds = mrf.predict(val)
mrf.score(trn, y_trn), mrf.score(val, y_val), mrf.oob_score_, exp_rmspe(preds, y_val)

CPU times: user 10min 13s, sys: 1.6 s, total: 10min 15s
Wall time: 1min 26s


(0.9822734429299947,
 0.9312465831110313,
 0.9248344812310892,
 0.10887143382612567)

# Deep Learning Model

In [12]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, yl.astype(np.float32), cat_flds=cat_vars, bs=128,
                                       test_df=df_test)
# Some categorical variables have a lot more levels than others. 
#   Store, in particular, has over a thousand!
cat_sz = [(c, len(joined_samp[c].cat.categories)+1) for c in cat_vars]

# We use the *cardinality* of each variable (that is, its number of unique values) to decide how 
#  large to make its *embeddings*. Each level will be associated with a vector with length defined as below.
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

# Inputs
#  Pass in No. of continuous variables =  
#       total Variables - Categoricals. 
#  NN knows how to create continuous and cat variables
#  Categorical variable dropouts ... .04
#  Output of last linear layer ... 1, predicting single number sales
#  Activations in first and second linear layers ... [1000,500]
#  Dropouts in first and second linear layers ... [.001,0.1]
# need to know embeddings matrix size
# in earlier models did not pass in the data ... in this case the model depends on the data
# the data object creates the learner
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

# Fit and Save the Model

In [13]:
%time m.fit(lr, 1, metrics=[exp_rmspe])

epoch      trn_loss   val_loss   exp_rmspe                       
    0      0.013347   0.021569   0.130493  

CPU times: user 44.3 s, sys: 2.26 s, total: 46.5 s
Wall time: 46.3 s


[array([0.02157]), 0.13049284647330175]

In [14]:
%time m.fit(lr, 3, metrics=[exp_rmspe])

epoch      trn_loss   val_loss   exp_rmspe                        
    0      0.010411   0.012753   0.11075   
    1      0.009634   0.012503   0.105795                         
    2      0.009381   0.013453   0.106811                         

CPU times: user 2min 13s, sys: 5.25 s, total: 2min 19s
Wall time: 2min 15s


[array([0.01345]), 0.10681116485543725]

In [15]:
%time m.fit(lr, 5, metrics=[exp_rmspe], cycle_len=1)

epoch      trn_loss   val_loss   exp_rmspe                        
    0      0.006373   0.010971   0.09824   
    1      0.006826   0.010797   0.097719                         
    2      0.006474   0.010548   0.09763                          
    3      0.006935   0.010639   0.097766                         
    4      0.006564   0.010407   0.096071                         

CPU times: user 3min 39s, sys: 9.54 s, total: 3min 49s
Wall time: 3min 43s


[array([0.01041]), 0.09607056865511707]

In [16]:
%time m.fit(lr, 4, metrics=[exp_rmspe], cycle_len=1)

epoch      trn_loss   val_loss   exp_rmspe                        
    0      0.006354   0.010429   0.095841  
    1      0.005756   0.010345   0.096118                         
    2      0.005986   0.010347   0.096728                         
    3      0.005627   0.010452   0.096509                         

CPU times: user 2min 56s, sys: 6.9 s, total: 3min 3s
Wall time: 2min 59s


[array([0.01045]), 0.09650949579673866]

In [13]:
m.save('val0')

# Test

In [18]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, yl.astype(np.float32), cat_flds=cat_vars, bs=128,
                                       test_df=df_test)
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
m.load('val0')

x,y=m.predict_with_targs()
print(exp_rmspe(x,y))

pred_test=m.predict(True)

pred_test = np.exp(pred_test)

pred_test

0.09979241514057972


array([[ 4475.422 ],
       [ 7242.9165],
       [ 9095.177 ],
       [ 7343.125 ],
       [ 7703.5986],
       [ 5933.637 ],
       [ 7425.5747],
       [ 8448.557 ],
       [ 5214.907 ],
       [ 6099.194 ],
       [ 7525.87  ],
       [ 8438.476 ],
       [ 7422.6226],
       [ 9582.    ],
       [ 6642.431 ],
       [ 4793.3516],
       [ 5958.497 ],
       [ 9812.533 ],
       [10718.219 ],
       [ 9720.873 ],
       ...,
       [ 7418.5815],
       [14485.122 ],
       [ 6268.0366],
       [ 5414.956 ],
       [ 8090.2046],
       [ 8611.748 ],
       [ 3226.5671],
       [ 8582.806 ],
       [ 6945.2866],
       [ 5872.405 ],
       [ 6023.3833],
       [ 5206.38  ],
       [ 3075.9094],
       [ 5696.65  ],
       [ 3826.3704],
       [ 2893.515 ],
       [ 7348.253 ],
       [ 6193.464 ],
       [24672.73  ],
       [ 7378.2866]], dtype=float32)

## Predict

In [15]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, yl.astype(np.float32), cat_flds=cat_vars, bs=128,
                                       test_df=df_test)
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
m.load('val0')

In [16]:
# columnar dataset
cds = ColumnarDataset.from_data_frame(df_test,cat_vars)
# data loader
dl = DataLoader(cds,batch_size=128)
# log predictions
predictions = m.predict_dl(dl)
# exp (log predictions) = predictions
predictions=np.exp(predictions)
predictions

array([[ 4475.422 ],
       [ 7242.9165],
       [ 9095.177 ],
       [ 7343.125 ],
       [ 7703.5986],
       [ 5933.637 ],
       [ 7425.5747],
       [ 8448.557 ],
       [ 5214.907 ],
       [ 6099.194 ],
       [ 7525.87  ],
       [ 8438.476 ],
       [ 7422.6226],
       [ 9582.    ],
       [ 6642.431 ],
       [ 4793.3516],
       [ 5958.497 ],
       [ 9812.533 ],
       [10718.219 ],
       [ 9720.873 ],
       ...,
       [ 7418.5815],
       [14485.122 ],
       [ 6268.0366],
       [ 5414.956 ],
       [ 8090.2046],
       [ 8611.748 ],
       [ 3226.5671],
       [ 8582.806 ],
       [ 6945.2866],
       [ 5872.405 ],
       [ 6023.3833],
       [ 5206.38  ],
       [ 3075.9094],
       [ 5696.65  ],
       [ 3826.3704],
       [ 2893.515 ],
       [ 7348.253 ],
       [ 6193.464 ],
       [24672.73  ],
       [ 7378.2866]], dtype=float32)

## RF

In [19]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
((val,trn), (y_val,y_trn)) = split_by_idx(val_idx, df.values, yl)
mrf = RandomForestRegressor(n_estimators=40, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)
mrf.fit(trn, y_trn);

preds = mrf.predict(val)
mrf.score(trn, y_trn), mrf.score(val, y_val), mrf.oob_score_, exp_rmspe(preds, y_val)

In [21]:
mrf.score(trn, y_trn), mrf.score(val, y_val), mrf.oob_score_, exp_rmspe(preds, y_val)

(0.9822761366872178,
 0.9315735953865742,
 0.9247582574628267,
 0.10865527725952048)